# Interface

### For Live Video

In [3]:
import cv2
import numpy as np
from keras.models import load_model
import datetime

# Current date and time for output filename
current_datetime = datetime.datetime.now()
current_date = current_datetime.strftime("%x").replace("/","")
current_time = current_datetime.strftime("%X").replace(":","")
output_filename = "output_"+str(current_date)+"_"+str(current_time)+".avi"

# Loading model
model=load_model("model2-009.model")

# Video overlay parameters
results={0:'without mask',1:'with mask'}
color_dict={0:(0,0,255),1:(0,255,0)}
rect_size = 4

# Initializing camera and output video writer
cap = cv2.VideoCapture(0) 
fourcc = cv2.VideoWriter_fourcc(*'DIVX') 
out = cv2.VideoWriter(output_filename, fourcc, 20.0, (640 , 480)) 

# Haarcascade face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while True:
    # Capturing image
    _, image = cap.read()
    image=cv2.flip(image,1,1)     
    rerect_size = cv2.resize(image, (image.shape[1] // rect_size, image.shape[0] // rect_size))
    
    # Detecting faces
    faces = face_classifier.detectMultiScale(rerect_size) 
    for face in faces:
        x, y, w, h = [v * rect_size for v in face] 
        
        # Image preprocessing for model
        face_img = image[y:y+h, x:x+w]
        face_img_resized=cv2.resize(face_img,(150,150))
        face_img_normalized=face_img_resized/255.0
        face_img_reshaped=np.reshape(face_img_normalized,(1,150,150,3))
        
        # Predicting image
        result=model.predict(face_img_reshaped)  
        
        # Plotting overlays
        label=np.argmax(result,axis=1)[0]
        cv2.rectangle(image,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(image,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(image, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    
    # Writing to output
    out.write(image)
    
    # Displaying video
    cv2.imshow('LIVE',image)
    key = cv2.waitKey(10)
    if key == 13: # Enter Key 
        break
        
# Releasing camera and video writer
cap.release()
out.release()
cv2.destroyAllWindows()

### For Single Image

In [30]:
from keras.models import load_model
import numpy as np
import cv2

# Video overlay parameters
rect_size = 4
results={0:'without mask',1:'with mask'}
color_dict={0:(0,0,255),1:(0,255,0)}

# Loading model
model = load_model('model2-009.model')

# Haarcascade face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Function to predict image, requires path to image
def predict_image(image_path):
    # Reading image
    image=cv2.imread(image_path) 
    rerect_size = cv2.resize(image, (image.shape[1] // rect_size, image.shape[0] // rect_size))
    
    # Detecting faces
    faces = face_classifier.detectMultiScale(rerect_size)
    
    for face in faces:
        x, y, w, h = [v * rect_size for v in face] 
        
        # Image preprocessing for model
        face_img = image[y:y+h, x:x+w]
        face_img_resized=cv2.resize(face_img,(150,150))
        face_img_normalized=face_img_resized/255.0
        face_img_reshaped=np.reshape(face_img_normalized,(1,150,150,3))
        
        # Predicting image
        result=model.predict(face_img_reshaped)
        
        # Plotting overlays
        label=np.argmax(result,axis=1)[0]
        cv2.rectangle(image,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(image,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(image, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    # Displaying image
    cv2.imshow('Single',image)
    
    # Saving operated image 
    output_image_name = image_path.split(".")[0]+"_output.jpg"
    cv2.imwrite(output_image_name,image)
    
    # Closing windows
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [31]:
# Predicting Image
predict_image('abhinav_without_mask.jpeg')
predict_image('abhinav_with_mask.jpeg')